# All classes
MedRoBERTa.nl – {“no label”: no label for all classes, 
                 “Normal”: normal for all classes,
                 “Present”: any non-normal present}
 

MedRoBERTa.nl – {“no label”: no label for all classes, 
                 “Normal”: normal for all classes,
                 “aortic_regurgitation_present”: ..,
                 “aortic_stenosis_present”: ..
                  …
                }
 
# Per class, i.e. 11 models
MedRoBERTa.nl – {“no label”: ..
                 “mild”: ..
                 “moderate”: ..
                 “severe”: ..
                “present”: ..}
 


%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import gc
from torchinfo import summary

from typing import List, Tuple, Dict, Union

In [ ]:
import torch

In [ ]:
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score

from transformers import TrainingArguments, Trainer
from transformers import pipeline
from transformers import DataCollatorWithPadding
from transformers import EvalPrediction

from torchinfo import summary
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

import benedict
import random

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("We will use the GPU:", torch.cuda.get_device_name(0))
    print(f"Memory Allocated: {torch.cuda.memory_allocated()}")
    print(f"Max memory Allocated: {torch.cuda.max_memory_allocated()}") 
    print(f"Memory reserved: {torch.cuda.memory_reserved()}")
    print(f"Max memory reserved: {torch.cuda.max_memory_reserved()}")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

In [ ]:
deabbreviate = False
filter_reports = True
Classes = ['mitral_regurgitation','tricuspid_regurgitation', 'wma'] # 'aortic_regurgitation'
FLAG_TERMS = ['uitslag zie medische status', 'zie status', 'zie verslag status', 'slecht echovenster', 'echo overwegen', 'ge echo',
              'geen echovenster', 'geen beoordeelbaar echo', 'tee 190', 'hdf 36mnd', 'geen beoordeelbare echo', 'verslag op ic', 'psyheartanalyse']
SAVE_TERMS = ['goed', 'geen', 'normaal', 'normale']
use_multilabel = False
reduce_labels = True
num_epochs = 5

MULTILABELS = {'Mild': ['Mild', 'Present'], 
               'Severe': ['Severe', 'Present'],
               'Moderate': ['Moderate', 'Present'],
               'Normal': ['Normal'],
               'No label': ['No label'],
               'Present': ['Present'],
               }
Target_remapper = {'No label': 'No label', 
                   'Normal': 'Normal',
                   'Mild': 'Not normal',
                   'Moderate': 'Not normal',
                   'Severe': 'Not normal',
                   'Present': 'Not normal'
                   }
#Target_remapper = {'No label': 'Normal', 
#                   'Normal': 'Normal',
#                   'Mild': 'Not normal',
#                   'Moderate': 'Not normal',
#                   'Severe': 'Not normal',
#                   'Present':  'Not normal'
#                   }

# train a MedRoBERTa.nl model that trains on
# No label: whether there is "no label" for all classes
# Normal: whether all the classes are normal
# Present: 


In [ ]:
#Add the src folder to the path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
import deabber, echo_utils

In [ ]:
if deabbreviate:
    ABBREVIATIONS = benedict.benedict("../assets/abbreviations.yml")

In [ ]:
plt.style.use('ggplot')
def plot_history(history, val=0):
    acc = history.history['accuracy']
    if val == 1:
        val_acc = history.history['val_accuracy'] # we can add a validation set in our fit function with nn
    loss = history.history['loss']
    if val == 1:
        val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training accuracy')
    if val == 1:
        plt.plot(x, val_acc, 'r', label='Validation accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.title('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    if val == 1:
        plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.title('Loss')
    plt.legend()

In [ ]:
def compute_metrics_binomial(logits_and_labels, averaging='macro'):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions == labels)
  f1 = f1_score(labels, predictions, average = averaging)
  prec = precision_score(labels, predictions, average = averaging)
  rec = recall_score(labels, predictions, average = averaging)
  return {
          'accuracy': acc, 
          'f1_score': f1,
          'precision': prec,
          'recal': rec
          }

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    
    
    f1_macro = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    f1_weighted = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
    prec_macro = precision_score(y_true=y_true, y_pred=y_pred, average='macro')
    prec_weighted = precision_score(y_true=y_true, y_pred=y_pred, average='weighted')
    recall_macro = recall_score(y_true=y_true, y_pred=y_pred, average='macro')
    recall_weighted = recall_score(y_true=y_true, y_pred=y_pred, average='weighted')    
    
    try:
        roc_auc_weighted = roc_auc_score(y_true, probs, average = 'weighted')
        roc_auc_macro = roc_auc_score(y_true, probs, average = 'macro')
    except:
        roc_auc_weighted = None
        roc_auc_macro = None
    
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1_macro': f1_macro,
               'f1_weighted': f1_weighted,
               'prec_macro': prec_macro,
               'prec_weighted': prec_weighted,
               'recall_macro': recall_macro,
               'recall_weighted': recall_weighted,
               'roc_auc_macro': roc_auc_macro,
               'roc_auc_weighted': roc_auc_weighted,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result
     


In [ ]:
def one_hot_encode(labels, num_classes):
    # labels should be a list or a 1D tensor
    one_hot = torch.zeros((len(labels), num_classes))
    rows = torch.arange(len(labels))
    one_hot[rows, labels] = 1
    return one_hot

def _mlabel_tuple_creator(x: List[int],
                          multilabels:Dict[int,List[int]],
                          num_classes: int=None)\
                          ->List[Tuple[int,...]]:
                              
    res = [(_sc for _sc in multilabels[sc]) for sc in x]
    return res

def multi_hot_encoding(x: List[int], 
                       multilabels: Union[Dict[int,List[int]], None]=None,
                       num_classes: int=None)\
                           ->torch.Tensor:    
    if multilabels is None:
        return one_hot_encode(x, num_classes=num_classes)
    else:
        return torch.Tensor(MultiLabelBinarizer(classes=range(num_classes))\
                    .fit_transform(_mlabel_tuple_creator(x,multilabels)))

# Load documents

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("CLTL/MedRoBERTa.nl")

In [ ]:
os.chdir('T://lab_research/RES-Folder-UPOD/Echo_label/E_ResearchData/2_ResearchData')

In [ ]:
for Class in Classes:
    labeled_documents = pd.read_json(f"./echo_doc_labels/{Class}.jsonl", lines=True)
    print(f"Label counts: {labeled_documents.label.value_counts()}")
    if reduce_labels:
        labeled_documents['label'] = labeled_documents.label.map(Target_remapper)
        print(f"Remapped labels: {labeled_documents.label.value_counts(dropna=False)}")
    # Expand with label columns
    Target_maps = {Label:i for i,Label in enumerate(labeled_documents['label'].unique())}
    
    # Load the train/test hashes
    test_hashes = pd.read_csv('./test_echoid.csv', sep=',')
    train_hashes = pd.read_csv('./train_echoid.csv', sep=',')
    print(train_hashes.columns)
    
    print(f"Train hashes: {train_hashes.input_hash.nunique()}")
    print(f"Test hashes: {test_hashes.input_hash.nunique()}")
    print(Target_maps.keys())
    
    # We now make DataSets (a special HuggingFace structure)
    # assuming cross-validation
    
    DF = labeled_documents
    DF.columns = ['sentence', 'labels', '_input_hash']
    
    label2id = Target_maps
    id2label = {v:k for k,v in label2id.items()}
    num_labels = len(label2id)
    DF['labels'] = DF['labels'].map(label2id)
    
    
    if filter_reports:
        DF = DF.assign(sentence = echo_utils.report_filter(DF.sentence, 
                                                flag_terms=FLAG_TERMS, 
                                                save_terms=SAVE_TERMS)[0])
        DF = DF.loc[DF.sentence.notna()]
    
    if deabbreviate:
        DeAbber = deabber.deabber(model_type='sbert', 
                                  abbreviations=ABBREVIATIONS['nl']['echocardiogram'], 
                                  min_sim=0.5, top_k=10)
        DF = DF.assign(sentence=DeAbber.deabb(DF.sentence.values, TokenRadius=3))
        
    if use_multilabel:
        _multilabels = {label2id[k]: [label2id[l] for l in v]
                        for k,v in MULTILABELS.items()}
    else:
        _multilabels = None
        
    
    # TODO: make proper
    DFtrain = DF.loc[DF._input_hash.isin(train_hashes.input_hash), ['sentence', 'labels']]
    DFtest = DF.loc[DF._input_hash.isin(test_hashes.input_hash), ['sentence', 'labels']]
    
    print("Train labels:")
    print(DFtrain.labels.value_counts())
    print("Test labels:")
    print(DFtest.labels.value_counts())
    
    TrainSet = Dataset.from_pandas(DFtrain)
    TestSet = Dataset.from_pandas(DFtest)
    
    HF_DataSet = DatasetDict(
        {'train' : TrainSet,
         'test': TestSet,
        }
    )
    
    Tokenized_DataSet = HF_DataSet.map(lambda batch: tokenizer(batch, truncation=True, 
                                                                      padding=True, 
                                                                      max_length=256),
                                      input_columns='sentence',
                                      batched=True,
                                      remove_columns=['sentence'])
    
    Tokenized_DataSet = (Tokenized_DataSet
                          #.map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
                          .map(lambda x: {"labels": 
                              multi_hot_encoding(x['labels'], 
                                                 multilabels=_multilabels, 
                                                 num_classes=num_labels)}, 
                               batched=True, remove_columns=['labels']))                      
                          #.rename_column("float_labels", "labels"))
    
    Tokenized_DataSet.set_format("torch", 
                                 columns=['input_ids', 'attention_mask', 'labels'])
    
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    
    # https://colab.research.google.com/drive/1aue7x525rKy6yYLqqt-5Ll96qjQvpqS7#scrollTo=1eVCRpcLUW-y
    # https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb
    medroberta_clf = AutoModelForSequenceClassification.from_pretrained("CLTL/MedRoBERTa.nl", 
                                                    num_labels=len(id2label.keys()),
                                                    problem_type='multi_label_classification',
                                                    id2label=id2label,
                                                    label2id=label2id)
    
    train_dir = "T:\\laupodteam/AIOS/Bram/data/tmp"
    metric_name = 'f1_macro'
    training_args = TrainingArguments(output_dir=train_dir,
                                      evaluation_strategy='epoch',
                                      save_strategy='epoch',
                                      num_train_epochs=num_epochs,
                                      learning_rate=5e-5,
                                      per_device_train_batch_size=16,
                                      weight_decay=0.01,
                                      per_device_eval_batch_size=10,
                                      load_best_model_at_end=True, 
                                      metric_for_best_model=metric_name)
    
    trainer = Trainer(medroberta_clf,
                      training_args,
                      train_dataset = Tokenized_DataSet["train"],
                      eval_dataset = Tokenized_DataSet["test"],
                      tokenizer=tokenizer,
                      data_collator=data_collator,
                      compute_metrics=compute_metrics)
    
    trainer.train()
    trainer.eval_dataset = Tokenized_DataSet["test"]
    _metrics = trainer.evaluate()
    
    
    reduced_label_str = "reduced_labels" if reduce_labels else "full_labels"
    mod_string = f"{Class}_Epochs{num_epochs}_{reduced_label_str}"
    new_dir = f"T://lab_research/RES-Folder-UPOD/Echo_label/G_Output/3_OtherOutput/MedRoBERTa/{mod_string}"
    os.mkdir(new_dir)
    trainer.save_model(new_dir)
    trainer.save_metrics(new_dir, metrics=_metrics)
    
    # clear GPU memory 
    del trainer, medroberta_clf    
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
summary(medroberta_clf)

In [ ]:
# consider quantisation using bitsandbytes or quanto